In [ ]:
%%html
<!-- P0 -->

<button type="button" class="btn btn-primary btn-md" onclick="initQB()">Initialise QB</button>
<script>
function initQB(){
    Jupyter.notebook.execute_cells([1]);
    Jupyter.notebook.execute_cells([2,3,4,5,6]);
}
</script>


In [ ]:
#P1

from pyspark.sql.functions import col
from pyspark.sql.window import Window
from pyspark.sql import Row, functions as F
from pyspark.sql.functions import unix_timestamp
from  pyspark.sql.functions import abs as cabs
import json
import requests
from hops import hdfs
import pydoop.hdfs as phdfs

myhdfs = phdfs.hdfs(user = hdfs.project_user())

tempDF = None
masterDF = None
hdfspath = "hdfs.project_path()"
g=None

def apply_filters(applied_filters):
    global tempDF
    applied_filters = json.loads(applied_filters)
    global g
    g = applied_filters
    filters_success = []
    for i in range(1,len(applied_filters)+1):
        index = str(i)
        filter_type = applied_filters[index]["type"]
        if applied_filters[index]["executed"]:
            continue        
        if filter_type=="filter":
            applied_filters[index]["parameters"][0] = "`"+applied_filters[index]["parameters"][0]+"`"
            if applied_filters[index]["parameters"][1]=="like":
                tempDF = tempDF.filter(applied_filters[index]["parameters"][0]+" "+applied_filters[index]["parameters"][1]+" "+applied_filters[index]["parameters"][2][:-1]+"%\'")
            else:
                try:
                    float(applied_filters[index]["parameters"][2])
                    tempDF = tempDF.filter(" ".join(applied_filters[index]["parameters"]))
                except:
                    applied_filters[index]["parameters"][2] = "'"+applied_filters[index]["parameters"][2]+"'"
                    tempDF = tempDF.filter(" ".join(applied_filters[index]["parameters"]))
        elif filter_type=="orderBy":
            paramsList = [x for x in applied_filters[index]["parameters"]]
            l = ["`"+x+"`" for x in list(paramsList[0])]
            tempDF = tempDF.orderBy(l,ascending = list(paramsList[1]))
        elif filter_type=="drop":
            tempDF = tempDF.drop(*applied_filters[index]["parameters"][0])
        elif filter_type=="withColumnRenamed":
            tempDF = tempDF.withColumnRenamed(applied_filters[index]["parameters"][0],applied_filters[index]["parameters"][1])
        elif filter_type=="select":
            l = ["`"+x+"`" for x in applied_filters[index]["parameters"][0]]
            tempDF = tempDF.select(*l)
        elif filter_type=="dropDuplicates":
            tempDF = tempDF.dropDuplicates()
        elif filter_type=="fillna":
            fillDict = dict(applied_filters[index]["parameters"][0])
            tempDF = tempDF.fillna(fillDict)
        elif filter_type=="replace":
            paramsList = [x for x in applied_filters[index]["parameters"]]
            tempDF = tempDF.replace(list(paramsList[0]),list(paramsList[1]),"`"+str(paramsList[2]+"`"))
        elif filter_type=="min":
            paramsList = [x for x in applied_filters[index]["parameters"]]
            colsp = list(paramsList[1])
            cols = ["`"+x+"`" for x in colsp]
            gby = ["`"+y+"`" for y in list(paramsList[0])]
            tempDF = tempDF.groupBy(gby).min(*cols)
        elif filter_type=="max":
            paramsList = [x for x in applied_filters[index]["parameters"]]
            colsp = list(paramsList[1])
            cols = ["`"+x+"`" for x in colsp]
            gby = ["`"+y+"`" for y in list(paramsList[0])]
            tempDF = tempDF.groupBy(gby).max(*cols)
        elif filter_type=="avg":
            paramsList = [x for x in applied_filters[index]["parameters"]]
            colsp = list(paramsList[1])
            cols = ["`"+x+"`" for x in colsp]
            gby = ["`"+y+"`" for y in list(paramsList[0])]
            tempDF = tempDF.groupBy(gby).avg(*cols)
        elif filter_type=="sum":
            paramsList = [x for x in applied_filters[index]["parameters"]]
            colsp = list(paramsList[1])
            cols = ["`"+x+"`" for x in colsp]
            gby = ["`"+y+"`" for y in list(paramsList[0])]
            tempDF = tempDF.groupBy(gby).sum(*cols)
        elif filter_type=="count":
            paramsList = [x for x in applied_filters[index]["parameters"]]
            l = ["`"+x+"`" for x in paramsList[0]]
            tempDF = tempDF.groupBy(list(l)).count()
        elif filter_type=="join":
            paramsList = [x for x in applied_filters[index]["parameters"]]
            keep = [x for x in paramsList[1:] if (None not in x)]
            jhow = str(paramsList[0])
            if len(keep) == 1:
                tempDF = masterDF.alias('masterDF').join(tempDF.alias('tempDF'),on=[col('masterDF.'+keep[0][0])==col('tempDF.'+keep[0][1])],how=jhow)
            elif len(keep) == 2:
                tempDF = masterDF.alias('masterDF').join(tempDF.alias('tempDF'),on=[col('masterDF.'+keep[0][0])==col('tempDF.'+keep[0][1]),col('masterDF.'+keep[1][0])==col('tempDF.'+keep[1][1])],how=jhow)
            elif len(keep) == 3:
                tempDF = masterDF.alias('masterDF').join(tempDF.alias('tempDF'),on=[col('masterDF.'+keep[0][0])==col('tempDF.'+keep[0][1]),col('masterDF.'+keep[1][0])==col('tempDF.'+keep[1][1]),col('masterDF.'+keep[2][0])==col('tempDF.'+keep[2][1])],how=jhow)
        elif filter_type=="joinApproxTime":
            g=1
            paramsList = [x for x in applied_filters[index]["parameters"]]
            keep = [x for x in paramsList[1:-1] if (None not in x)]
            jhow = str(paramsList[0])
            time_columns = paramsList[-1]
            g=2
            allcols = tempDF.columns
            allcols.extend(masterDF.columns)
            g=3
            if len(keep) == 1:
                g=4
                joinedDF = masterDF.alias('masterDF').join(tempDF.alias('tempDF'),on=[col('masterDF.'+keep[0][0])==col('tempDF.'+keep[0][1])],how=jhow)
                additional_col = joinedDF.withColumn("time_diff",  cabs(unix_timestamp(str(time_columns[0])) - unix_timestamp(str(time_columns[1]))))
                partDF = additional_col.select(F.row_number().over(Window.partitionBy(str(keep[0][0])).orderBy("time_diff")).alias("rank"), "time_diff",*allcols ).filter(col("rank") == 1)
                tempDF = partDF
            elif len(keep) == 2:
                g=5
                joinedDF = masterDF.alias('masterDF').join(tempDF.alias('tempDF'),on=[col('masterDF.'+keep[0][0])==col('tempDF.'+keep[0][1]),col('masterDF.'+keep[1][0])==col('tempDF.'+keep[1][1])],how=jhow)
                additional_col = joinedDF.withColumn("time_diff",  cabs(unix_timestamp(str(time_columns[0])) - unix_timestamp(str(time_columns[1]))))
                partDf = additional_col.select( F.row_number().over(Window.partitionBy(str(keep[0][0]),str(keep[1][0])).orderBy("time_diff")).alias("rank"), *allcols ).filter(col("rank") == 1).drop('rank')
                tempDF = partDF
            elif len(keep) == 3:
                g=6
                joinedDF = masterDF.alias('masterDF').join(tempDF.alias('tempDF'),on=[col('masterDF.'+keep[0][0])==col('tempDF.'+keep[0][1]),col('masterDF.'+keep[1][0])==col('tempDF.'+keep[1][1]),col('masterDF.'+keep[2][0])==col('tempDF.'+keep[2][1])],how=jhow)
                additional_col = joinedDF.withColumn("time_diff",  cabs(unix_timestamp(str(time_columns[0])) - unix_timestamp(str(time_columns[1]))))
                partDf = additional_col.select( F.row_number().over(Window.partitionBy(str(keep[0][0]),str(keep[1][0]),str(keep[2][0])).orderBy("time_diff")).alias("rank"), *allcols).filter(col("rank") == 1).drop('rank')
                tempDF = partDF
        filters_success.append(index)
    return filters_success
            
#z.put("applied_filters",applied_filters)
#z.put('tempDF_cols',','.join(tempDF.columns))
#z.put('tempDF_cols',",".join(tempDF.columns))

def read_dataset(selectedFile):
    global tempDF
    tempDF = spark.read.csv(selectedFile,header=True, inferSchema=True)
    return tempDF.columns

def tempDFtoMasterDF():
    global masterDF
    global tempDF
    masterDF = tempDF
    return masterDF.columns

def saveMasterAsCSV(dirname):
    global masterDF
    global hdfspath
    masterDF.coalesce(1).write.csv(path=hdfspath+'/'+dirname, mode='append', header=True)
    return 200

def getCSVfiles():
    fileSet = set()
    for f in myhdfs.walk(hdfs.project_path()):
        fileSet.add(f['name'])
    available_files = [x for x in list(fileSet) if x.endswith(".csv")]
    return available_files

def get_datasets():
    r2 = requests.get("http://aegis-metadata.fokus.fraunhofer.de/api/datasets")
    return r2.text

def get_dataset(name):
    r2 = requests.get("http://aegis-metadata.fokus.fraunhofer.de/api/datasets/"+name+"/distributions")
    return r2.text

#TODO: use Fabian's info on columns...
def read_dataset_fromMetadata(selectedFile):
    global tempDF
    try:
        tempDF = spark.read.csv(selectedFile,header=True, inferSchema=True)
        return tempDF.columns
    except:
        return 500

In [ ]:
%%html
<!-- P2 -->

<style>

.tagclass {
    background:grey;
    border:none;
}
.fieldcell{
    white-space:pre-wrap;
}
td{
  text-align:left;
}

table{
    table-layout: auto !important;
}

td{
    word-wrap:break-word
}


.paragraph .form-horizontal .form-group {
    margin-right: 0px !important;
    margin-left: 0px !important;
}

#list2 { width:320px; }
#list2 ol { font-style:italic; font-family:Georgia, Times, serif; font-size:18px; color:#337ab7;  }
#list2 ol li { }
#list2 ol li p { padding:8px; font-style:normal; font-family:Arial; font-size:13px; color:black; border-left: 1px solid #999; }
#list2 ol li.applied p em { color:lightgray;  }
#list2 ol li p em { display:block; }


.qblabel {
    font-style:italic; font-family:Georgia, Times, serif; font-size:18px; color:#337ab7; 
}

</style>
<button type="button" class="btn btn-primary btn-md" onclick="showCurrentBrowser()">Local Broswer</button>
<button type="button" class="btn btn-primary btn-md" onclick="showFutureBrowser();">Metadata Browser</button>
<hr><hr>
<div id="metadata-datasets" class="panel panel-default" style="display:none">

  <div class="panel-body">
  <table class="table">
    <thead>
      <tr>
        <th data-field="title">Title</th>
        <th data-field="description">Description</th>
        <th data-field="contact-point">Contact Point</th>
        <th data-field="publisher">Publisher</th>
        <th data-field="keywords">Keywords</th>
      </tr>
    </thead>
    <tbody id="fab2">
    </tbody>
  </table>
  
  </div>
</div>

<div id="metadata-datasets-info" class="panel panel-default" style="display:none">

  <div class="panel-body">
  <table class="table">
    <thead>
      <tr>
        <th data-field="loadlink"></th>
        <th data-field="title">Title</th>
        <th data-field="description">Description</th>
        <th data-field="license">License</th>
        <th data-field="fields">Fields</th>
      </tr>
    </thead>
    <tbody id="datasets-info-table">
    </tbody>
  </table>
  
  </div>
</div>

<div id="cbrowser" style="display:none">
  <div class="form-group">
              <label class="qblabel">Available files</label>
              <div>
                <select id="fileSelect" class="form-control" >
                  
                </select> 
              </div>
            </div>
            <button type="button" class="btn btn-primary btn-md" onclick="selectDataset()"> Select </button>
</div>
<hr><hr>
<div>
<div class="form-horizontal" id="null-filterType" style="display:none;">
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Choose action</label>
        <div class="col-sm-10">
            <select class="form-control" id="filterType" onchange="divVisibilityChanged()" >
                <option value=""></option>
                <option value="filter">Filter</option>
                <option value="orderBy">Order By</option>
                <option value="count">Count</option>
                <option value="min">Min</option>
                <option value="max">Max</option>
                <option value="avg">Average</option>
                <option value="sum">Sum</option>
                <option value="select">Select column(s)</option>
                <option value="drop">Drop column(s)</option>
                <option value="fillna">Fill N/A</option>
                <option value="replace">Replace value</option>
                <option value="withColumnRenamed">Rename Column</option>
                <option value="join">Join with master</option>
                <option value="joinApproxTime">Join with master on approximate time</option>
            </select>
        </div>
    </div>
    <!--
    <div class="form-group">
        <div class="col-sm-offset-2 col-sm-10">
            <button class="btn btn-primary" id="filters-done" onclick="applyAllFilters();"> Done </button>
            <button class="btn btn-danger" id="reset-filters" onclick="resetAllFilters();"> Reset </button>
        </div>
    </div>
    -->
</div>
<div class="form-horizontal" id="filterType_filter" style="display:none" >
    <h3>Filter</h3>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Column</label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector temp-column" >
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Operator</label>
        <div class="col-sm-10">
            <select class="form-control operator-selector" >
                <option></option>
                <option value="<">&lt;</option>
                <option value="==">=</option>
                <option value=">">&gt;</option>
                <option value="!=">!=</option>
                <option value="like">starts with</option>
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Value</label>
        <div class="col-sm-10">
            <input class="form-control user-defined-value" />
        </div>
    </div>
</div>

<div class="form-horizontal" id="filterType_orderBy" style="display:none" >
    <h3>Order By</h3>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Column</label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector temp-column" >
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Sort</label>
        <div class="col-sm-10">
            <select class="form-control operator-selector">
                <option value="1">Asc</option>
                <option value="0">Desc</option>
            </select>
        </div>
    </div>
</div>

<div class="form-horizontal" id="filterType_count" style="display:none">
    <h3>Count</h3>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Group By</label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector temp-column"  >
            </select>
        </div>
    </div>
</div>

<div class="form-horizontal" id="filterType_min" style="display:none">
    <h3>Min</h3>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Columns</label>
        <div class="col-sm-10">
            <select multiple class="form-control multiple-column-selector temp-column" >
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Group By</label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector temp-column"  >
            </select>
        </div>
    </div>

</div>

<div class="form-horizontal" id="filterType_max" style="display:none">
    <h3>Max</h3>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Columns</label>
        <div class="col-sm-10">
            <select multiple class="form-control multiple-column-selector temp-column" >
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Group By</label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector temp-column" >
            </select>
        </div>
    </div>
</div>

<div class="form-horizontal" id="filterType_avg" style="display:none">
    <h3>Average</h3>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Columns</label>
        <div class="col-sm-10">
            <select multiple class="form-control multiple-column-selector temp-column" >
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Group By</label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector temp-column" >
            </select>
        </div>
    </div>
</div>

<div class="form-horizontal" id="filterType_sum" style="display:none" >
    <h3>Sum</h3>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Columns</label>
        <div class="col-sm-10">
            <select multiple class="form-control multiple-column-selector temp-column" >                
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Group By</label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector temp-column" >
            </select>
        </div>
    </div>
</div>


<div class="form-horizontal" id="filterType_select" style="display:none" >
    <h3>Select Columns</h3>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Columns</label>
        <div class="col-sm-10">
            <select multiple class="form-control multiple-column-selector temp-column" >
            </select>
        </div>
    </div>
</div>

<div class="form-horizontal" id="filterType_drop" style="display:none">
    <h3>Select Columns</h3>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Columns</label>
        <div class="col-sm-10">
            <select multiple class="form-control multiple-column-selector temp-column" >
            </select>
        </div>
    </div>
</div>

<div class="form-horizontal" id="filterType_fillna" style="display:none" >
    <h3>Fill N/A</h3>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Column</label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector temp-column" >
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Value</label>
        <div class="col-sm-10">
            <input class="form-control user-defined-value" />
        </div>
    </div>
</div>

<div class="form-horizontal" id="filterType_replace" style="display:none" >
    <h3>Replace Value</h3>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Column</label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector temp-column" >
                <option></option>
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Current Value</label>
        <div class="col-sm-10">
            <input class="form-control user-defined-value existing-value" />
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">To-Be Value</label>
        <div class="col-sm-10">
            <input class="form-control user-defined-value to-be-value" />
        </div>
    </div>
</div>

<div class="form-horizontal" id="filterType_withColumnRenamed" style="display:none" >
    <h3>Rename Column</h3>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Column</label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector temp-column" >
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">New Name</label>
        <div class="col-sm-10">
            <input class="form-control user-defined-value" />
        </div>
    </div>
</div>

<div class="form-horizontal" id="filterType_join" style="display:none" >
    <h3>Join</h3>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Join Type</label>
        <div class="col-sm-10">
            <select class="form-control operator-selector" >
                <option value="inner">inner</option>
                <option value="outer">outer</option>
                <option value="left_outer">left_outer</option>
                <option value="right_outer">right_outer</option>
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Master Column 1</label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector master-column master-col-1">
            <option></option>
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Temp Column 1</label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector temp-col-1 temp-column empty-option-allowed" >
            <option></option>
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Master Column 2 (opt) </label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector master-column master-col-2" >
            <option></option>
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Temp Column 2 (opt) </label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector temp-col-2 temp-column empty-option-allowed"  >
            <option></option>
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Master Column 3 (opt) </label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector master-column master-col-3" >
            <option></option>
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Temp Column 3 (opt) </label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector temp-col-3 temp-column empty-option-allowed" >
            <option></option>
            </select>
        </div>
    </div>
</div>

<div class="form-horizontal" id="filterType_joinApproxTime" style="display:none">
    <h3>Join on closest time</h3>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Join Type</label>
        <div class="col-sm-10">
            <select class="form-control operator-selector" >
                <option value="inner">inner</option>
                <option value="outer">outer</option>
                <option value="left_outer">left_outer</option>
                <option value="right_outer">right_outer</option>
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Master Time Column</label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector master-column master-time-col" >
                <option></option>
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Temp Time Column</label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector temp-time-col temp-column empty-option-allowed" >
                <option></option>
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Master Column 1</label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector master-column master-col-1" >
            <option></option>
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Temp Column 1</label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector temp-col-1 temp-column empty-option-allowed" >
            <option></option>
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Master Column 2 (opt) </label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector master-column master-col-2" >
            <option></option>
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Temp Column 2 (opt) </label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector temp-col-2 temp-column empty-option-allowed" >
            <option></option>
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Master Column 3 (opt) </label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector master-column master-col-3" >
            <option></option>
            </select>
        </div>
    </div>
    <div class="form-group">
        <label class="control-label col-sm-2" for="file">Temp Column 3 (opt) </label>
        <div class="col-sm-10">
            <select class="form-control single-column-selector temp-col-3 temp-column empty-option-allowed" >
            <option></option>
            </select>
        </div>
    </div>
</div>

<div id="filter-control-buttons" style="display:none">
        <div class="col-sm-offset-2 col-sm-10">
            <button class="btn btn-primary btn-sm" onclick="addFilter();"> OK </button>
            <button class="btn btn-warning btn-sm" onclick="cancelFilter()"> Cancel </button>
        </div>
        <hr><hr>

    </div>

<div id="filter-application-buttons" style="display:none" align="right">
<button type="button" class="btn btn-success btn-sm" onclick="applyAllFilters();"> Apply Filters </button>
<button type="button" class="btn btn-success btn-sm" onclick="Jupyter.notebook.execute_cells([4]);"> Refresh temp </button>
<button type="button" class="btn btn-danger btn-sm" onclick="tempToMaster();">Save temp as master</button>
<button type="button" class="btn btn-success btn-sm" onclick="Jupyter.notebook.execute_cells([6]);"> Refresh master </button>
<button type="button" class="btn btn-danger btn-sm" onclick="exportMaster();">Save master to CSV</button>
</div>

<div id="list2"><ol id="show-filters"></ol></div>



<script>

var tempDF_columns = []
var datafile_options = []
var filter_divs = ["null-filterType","filterType_joinApproxTime","filterType_join","filterType_withColumnRenamed","filterType_replace","filterType_fillna","filterType_orderBy","filterType_count","filterType_min","filterType_max","filterType_sum","filterType_avg","filterType_filter","filterType_select","filterType_drop"]
var visibleDivId = null;
var applied_filters = {}
var filter_count = 0
var successfulFilters=[]
var successfulFiltersPermanent = [] <!-- temporary solution, to be removed -->

function show_filters(){
    $("#show-filters").empty()
    $("#show-filters").append("<label class='qblabel'>Selected Filters</label>")
    $.each( applied_filters, function( key, val ) {
        if(successfulFiltersPermanent.includes(key)){
            $("#show-filters").append( "<li id='" + key + "' class='applied'><p><em>" + val["descr"] + "</em></p></li>" );
        }
        else{
            $("#show-filters").append( "<li id='" + key + "' class='notapplied'><p><em>" + val["descr"] + "</em></p></li>" );
        }
    });
}

function exportMaster(){
    var kernel = IPython.notebook.kernel;
    var callbacks = {
        iopub : {
             output : csvExported,
        }
    }
    dataset = document.getElementById('fileSelect').value;
    var msg_id = kernel.execute('saveMasterAsCSV("resultsdir")', callbacks);

}

<!-- call python to read file from hdfs, reset filter dropdown and populate column dropdowns -->
function selectDataset(){
    var kernel = IPython.notebook.kernel;
    var callbacks = {
        iopub : {
             output : handle_output,
        }
    }
    dataset = document.getElementById('fileSelect').value;
    var msg_id = kernel.execute('read_dataset("'+dataset+'")', callbacks);
    divVisibility("null-filterType");
    document.getElementById("filter-application-buttons").style.display = "block"; //because now we have a tempDF
    visibleDivId = null;
    applied_filters = {}
    filter_count = 0
    $("#show-filters").empty()
    successfulFilters = [];
    successfulFiltersPermanent = [];
}

function getCSVfilesFromFS(data){
    datafile_options = data.content.text;
    datasetsObtained(JSON.parse(datafile_options.replace(/u\'/g,'"').replace(/\'/g,'"')))
}

<!-- add tempDF columns to the dropdown selectors -->
function handle_output(data){
    tempDF_columns = data.content.text;
    addColumnFields(tempDF_columns);
}

function filters_applied_result(data){
    successfulFilters = JSON.parse(data.content.text.replace(/\'/g,'"'))
    for (var i = 0; i < successfulFilters.length; i++) {
        var index = successfulFilters[i];
        applied_filters[index]["executed"] = true;
        successfulFiltersPermanent.push(index);
    }
    show_filters();
}

function tempToMaster(){
    var kernel = IPython.notebook.kernel;
    var callbacks = {
        iopub : {
             output : masterDFupdated,
        }
    }
    var msg_id = kernel.execute('tempDFtoMasterDF()', callbacks);
}

function csvExported(){
    alert("CSV ready")
}                                                             
                                                             
function masterDFupdated(data){
    masterDF_columns = data.content.text;
    addMasterColumnFields(masterDF_columns);
    alert("masterDFchanged")
}
                                                             
function showFutureBrowser() {
    var x = document.getElementById("metadata-datasets");
    if (x.style.display === "none") {
        document.getElementById("cbrowser").style.display = "none";
        x.style.display = "block";
        retrieve_datasets();
    } else {
        x.style.display = "none";
    }
} 
function showCurrentBrowser() {
    var x = document.getElementById("cbrowser");
    if (x.style.display === "none") {
        var kernel = IPython.notebook.kernel;
        var callbacks = {
            iopub : {
                output : getCSVfilesFromFS,
            }
        }
        var msg_id = kernel.execute('getCSVfiles()', callbacks);
    } else {
        x.style.display = "none";
    }
} 
                                                             
function datasetsObtained(options){
    var x = document.getElementById("cbrowser");
    var dropl = document.getElementById("fileSelect");
    dropl.options.length = 0;
    for (opt in options) {
        dropl.options[dropl.options.length] = new Option(options[opt]);
    }
    if (x.style.display === "none") {
        document.getElementById("metadata-datasets").style.display = "none";
        x.style.display = "block";
    } else {
        x.style.display = "none";
    }
}                                                             

function divVisibilityChanged() {
    enabled_filter = document.getElementById("filterType").value;
    divid = "filterType_".concat(enabled_filter);
    divVisibility(divid);
}

<!-- choose which of the forms will be visible -->
function divVisibility(divId) {
    visibleDivId = divId;
    hideNonVisibleDivs();
    
    <!--
    if(visibleDivId === divId) {
        visibleDivId = null;
      } 
    else {
        visibleDivId = divId;
      }
    -->
}

<!-- helper:called to choose which of the forms will be visible -->
function hideNonVisibleDivs() {
      var i, divId, div;
      for(i = 0; i < filter_divs.length; i++) {
        divId = filter_divs[i];
        div = document.getElementById(divId);
        if(visibleDivId === divId) {
          div.style.display = "block";
        } else {
          div.style.display = "none";
        }
      }
    if(visibleDivId != null){
        document.getElementById("filter-control-buttons").style.display = "block";
    }
    else{
        document.getElementById("filter-control-buttons").style.display = "none";
    }
    
}

<!-- populate column dropdowns -->
function addColumnFields(columns){
    arraycols = JSON.parse(columns.replace(/\'/g,'"'))
    $('.temp-column').each(function(index){
        if($(this).hasClass("empty-option-allowed")){
            $(this).children('option:not(:first)').remove();
        }
        else{
            $(this).children('option').remove();
        }
        
        for(var i = 0; i < arraycols.length; i++){
            optionGenerator($(this),arraycols[i].trim());
        }
        
    });
    
}
                                           
<!-- populate master column dropdowns -->
function addMasterColumnFields(columns){
    masterarraycols = JSON.parse(columns.replace(/\'/g,'"'))
    $('.master-column').each(function(index){
        $(this).children('option:not(:first)').remove();
        for(var i = 0; i < masterarraycols.length; i++){
            optionGenerator($(this),masterarraycols[i].trim());
        }
        
    });                                             
}

function applyAllFilters(){
    var kernel = IPython.notebook.kernel;
    var callbacks = {
        iopub : {
             output : filters_applied_result,
        }
    }
    agh = (JSON.stringify(applied_filters))
    lala = {}
    var msg_id = kernel.execute('apply_filters(\''+agh+'\')', callbacks);
}

function cancelFilter(){
    divVisibility('null-filterType');
    $("#filterType")[0].selectedIndex = 0;    
}

function addFilter(){
    enabled_filter = document.getElementById("filterType").value;
    filterObj = {}
    filter_count++;
    switch(enabled_filter){
        case "filter":{
            filterObj["type"] = enabled_filter
            filterObj["executed"] = false
            parameters = []
            parameters.push($("#filterType_filter .single-column-selector").find(":selected").val())
            parameters.push($("#filterType_filter .operator-selector").find(":selected").val())
            parameters.push($("#filterType_filter .user-defined-value").val())
            filterObj["parameters"] = parameters
            filterObj["descr"] = parameters.toString();
            applied_filters[filter_count.toString()] = filterObj;
            break;
        }
        case "orderBy":{
            filterObj["type"] = enabled_filter
            filterObj["executed"] = false
            parameters = []
            parameters.push([$("#filterType_orderBy .single-column-selector").find(":selected").val()])
            parameters.push([Number($("#filterType_orderBy .operator-selector").find(":selected").val())])
            filterObj["parameters"] = parameters
            filterObj["descr"] = parameters[0][0] + '(' + $("#filterType_orderBy .operator-selector").find(":selected").text() + ')'
            applied_filters[filter_count.toString()] = filterObj;
            break;
        }
        case "count":{
            filterObj["type"] = enabled_filter
            filterObj["executed"] = false
            parameters = []
            parameters.push([$("#filterType_count .single-column-selector").find(":selected").val()])
            filterObj["parameters"] = parameters
            filterObj["descr"] = "Count [OrderBy:"+parameters[0][0]+"]"
            applied_filters[filter_count.toString()] = filterObj;
            break;
        }
        case "select":{
            filterObj["type"] = enabled_filter
            filterObj["executed"] = false
            parameters = []
            param_array = $("#filterType_select .multiple-column-selector option:selected").map(function(){ return this.value }).get();
            parameters.push(param_array);
            filterObj["parameters"] = parameters
            filterObj["descr"] = "Select ["+parameters.toString()+"]"
            applied_filters[filter_count.toString()] = filterObj;
            break;
        }
        case "drop":{
            filterObj["type"] = enabled_filter
            filterObj["executed"] = false
            parameters = []
            param_array = $("#filterType_drop .multiple-column-selector option:selected").map(function(){ return this.value }).get();
            parameters.push(param_array);
            filterObj["parameters"] = parameters
            filterObj["descr"] = "Drop ["+parameters.toString()+"]"
            applied_filters[filter_count.toString()] = filterObj;
            break;
        }
        case "min":{
            filterObj["type"] = enabled_filter
            filterObj["executed"] = false
            parameters = []
            parameters.push([$("#filterType_min .single-column-selector").find(":selected").val()])
            cols_array = $("#filterType_min .multiple-column-selector option:selected").map(function(){ return this.value }).get();
            parameters.push(cols_array);
            filterObj["parameters"] = parameters
            filterObj["descr"] = "Min ("+parameters[1].toString()+") - Group by "+parameters[0][0]
            applied_filters[filter_count.toString()] = filterObj;
            break;
        }
        case "max":{
            filterObj["type"] = enabled_filter
            filterObj["executed"] = false
            parameters = []
            parameters.push([$("#filterType_max .single-column-selector").find(":selected").val()])
            cols_array = $("#filterType_max .multiple-column-selector option:selected").map(function(){ return this.value }).get();
            parameters.push(cols_array);
            filterObj["parameters"] = parameters
            filterObj["descr"] = "Max ("+parameters[1].toString()+") - Group by "+parameters[0][0]
            applied_filters[filter_count.toString()] = filterObj;
            break;
        }
        case "sum":{
            filterObj["type"] = enabled_filter
            filterObj["executed"] = false
            parameters = []
            parameters.push([$("#filterType_sum .single-column-selector").find(":selected").val()])
            cols_array = $("#filterType_sum .multiple-column-selector option:selected").map(function(){ return this.value }).get();
            parameters.push(cols_array);
            filterObj["parameters"] = parameters
            filterObj["descr"] = "Sum ("+parameters[1].toString()+") - Group by "+parameters[0][0]
            applied_filters[filter_count.toString()] = filterObj;
            break;
        }
        case "avg":{
            filterObj["type"] = enabled_filter
            filterObj["executed"] = false
            parameters = []
            parameters.push([$("#filterType_avg .single-column-selector").find(":selected").val()])
            cols_array = $("#filterType_avg .multiple-column-selector option:selected").map(function(){ return this.value }).get();
            parameters.push(cols_array);
            filterObj["parameters"] = parameters
            filterObj["descr"] = "Average ("+parameters[1].toString()+") - Group by "+parameters[0][0]
            applied_filters[filter_count.toString()] = filterObj;
            break;
        }
        case "fillna":{
            filterObj["type"] = enabled_filter
            filterObj["executed"] = false
            parameters = []
            pdict={}
            col = $("#filterType_fillna .single-column-selector").find(":selected").val();
            newv = $("#filterType_fillna .user-defined-value").val();
            pdict[col] = newv
            parameters.push(pdict)
            filterObj["parameters"] = parameters
            filterObj["descr"] = col + ":Fill n/a with "+newv;
            applied_filters[filter_count.toString()] = filterObj;
            break;
        }
        case "replace":{
            filterObj["type"] = enabled_filter
            filterObj["executed"] = false
            parameters = []
            parameters.push([$("#filterType_replace .user-defined-value.existing-value").val()])
            parameters.push([$("#filterType_replace .user-defined-value.to-be-value").val()])
            parameters.push($("#filterType_replace .single-column-selector").find(":selected").val())
            filterObj["parameters"] = parameters
            filterObj["descr"] = parameters[2] + " [" + parameters[0][0] + " --> " + parameters[1][0] + "]"
            applied_filters[filter_count.toString()] = filterObj;
            break;
        }
        case "withColumnRenamed":{
            filterObj["type"] = enabled_filter
            filterObj["executed"] = false
            parameters = []
            parameters.push($("#filterType_withColumnRenamed .single-column-selector").find(":selected").val())
            parameters.push($("#filterType_withColumnRenamed .user-defined-value").val())
            filterObj["parameters"] = parameters
            filterObj["descr"] = "Rename " + parameters[0] + " to " + parameters[1]
            applied_filters[filter_count.toString()] = filterObj;
            break;
        }
        case "join":{
            filterObj["type"] = enabled_filter
            filterObj["executed"] = false
            parameters = []
            operator = $("#filterType_join .operator-selector").find(":selected").val()
            tempCol1 = $("#filterType_join .single-column-selector.temp-col-1").find(":selected").val()
            tempCol2 = $("#filterType_join .single-column-selector.temp-col-2").find(":selected").val() || null
            tempCol3 = $("#filterType_join .single-column-selector.temp-col-3").find(":selected").val() || null
            masterCol1 = $("#filterType_join .single-column-selector.master-col-1").find(":selected").val()
            masterCol2 = $("#filterType_join .single-column-selector.master-col-2").find(":selected").val() || null
            masterCol3 = $("#filterType_join .single-column-selector.master-col-3").find(":selected").val() || null
            parameters.push(operator)
            parameters.push([masterCol1, tempCol1])
            parameters.push([masterCol2, tempCol2])
            parameters.push([masterCol3, tempCol3])
            filterObj["parameters"] = parameters
            filterObj["descr"] = operator + "join: (" + masterCol1 + ","+ tempCol1 + ")" + "(" + masterCol2 + "," + tempCol2 + ")" +"(" + masterCol3 + "," + tempCol3 + ")"
            applied_filters[filter_count.toString()] = filterObj;
            break;
        }
        case "joinApproxTime":{
            filterObj["type"] = enabled_filter
            filterObj["executed"] = false
            parameters = []
            operator = $("#filterType_joinApproxTime .operator-selector").find(":selected").val()
            tempTimeCol = $("#filterType_joinApproxTime .single-column-selector.temp-time-col").find(":selected").val()
            masterTimeCol = $("#filterType_joinApproxTime .single-column-selector.master-time-col").find(":selected").val()
            tempCol1 = $("#filterType_joinApproxTime .single-column-selector.temp-col-1").find(":selected").val()
            tempCol2 = $("#filterType_joinApproxTime .single-column-selector.temp-col-2").find(":selected").val() || null
            tempCol3 = $("#filterType_joinApproxTime .single-column-selector.temp-col-3").find(":selected").val() || null
            masterCol1 = $("#filterType_joinApproxTime .single-column-selector.master-col-1").find(":selected").val()
            masterCol2 = $("#filterType_joinApproxTime .single-column-selector.master-col-2").find(":selected").val() || null
            masterCol3 = $("#filterType_joinApproxTime .single-column-selector.master-col-3").find(":selected").val() || null
            parameters.push(operator)
            parameters.push([masterCol1, tempCol1])
            parameters.push([masterCol2, tempCol2])
            parameters.push([masterCol3, tempCol3])
            parameters.push([masterTimeCol,tempTimeCol])
            filterObj["parameters"] = parameters
            filterObj["descr"] = operator + "approx time join on ["+masterTimeCol + ","+ tempTimeCol+"]: (" + masterCol1 + ","+ tempCol1 + ")" + "(" + masterCol2 + "," + tempCol2 + ")" +"(" + masterCol3 + "," + tempCol3 + ")"
            applied_filters[filter_count.toString()] = filterObj;
            break;
        }
    }
    show_filters();
    divVisibility('null-filterType');
    $("#filterType")[0].selectedIndex = 0;

}

function resetAllFilters(){
    applied_filters = []
    filter_count = 0
}

<!-- helper: adds options to help populate column dropdowns -->
function optionGenerator(instance,paramObj){  
    instance
    .append($("<option></option>")
            .attr("value",paramObj)
            .text(paramObj));
};
</script>
                                                 
                                                 <script>
var k=0;
function handle_datasets_output(data){
    available_datasets = data.content;
    available_datasets = JSON.parse(available_datasets.text.slice(2,-1));
    $("#fab2").empty();
    
    $.each( available_datasets["results"], function( key, val ) {
        var keywords = ""
        for(i = 0; i < val["keywords"].length; i++) {
            keywords = keywords + ("<span class='label label-default'>"+val["keywords"][i]+"</span>");
        }
        $("#fab2").append("<tr>\
        <td><button type='button' class='btn btn-primary btn-xs' style='width:100%' onclick='getRequestedDataset(\""+val["name"]+"\");'>"+val["name"]+"</button></td>\
        <td>" + val["description"]+"</td>\
        <td>" + val["contact_point"]["name"]+" ("+val["contact_point"]["email"]+")</td>\
        <td><a href='"+ val["publisher"]["homepage"]+"'>"+val["publisher"]["name"]+"</a></td>\
        <td>" + keywords + "</td>\
        </tr>");
    });
    $("#metadata-datasets").show();
}

function handle_dataset_retrieval(data){
    dataset = data.content;
    dataset = JSON.parse(dataset.text.slice(2,-1));
    $("#datasets-info-table").empty();
    
    $.each( dataset["results"], function( key, val ) {
        var fields = ""
        for(i = 0; i < val["fields"].length; i++) {
            fields = fields + val["fields"][i]["name"] + "(" + val["fields"][i]["type"] + "):" + val["fields"][i]["description"] + "<br>"
        }
        $("#datasets-info-table").append("<tr>\
        <td><button type='button' class='btn btn-primary btn-xs' style='width:100%' onclick='readRequestedDataset(\""+val["access_url"]+"\");'>Open</button></td>\
        <td>" + val["title"]+"</td>\
        <td>" + val["description"]+"</td>\
        <td>" + val["license"]+"</td>\
        <td class='fieldcell'>" + fields + "</td>\
        </tr>");
    });
    $("#metadata-datasets-info").show();
    $("#metadata-datasets").hide();
}

function handle_dataset_loading(data){
    response = data.content.text;
    if(response=="500"){
        alert("Oops! Something went wrong: cannot read file.")
    }
    else{
        addColumnFields(tempDF_columns);
    }
}

function retrieve_datasets(){
    var kernel = IPython.notebook.kernel;
    var callbacks = {
        iopub : {
             output : handle_datasets_output,
        }
    }
    var msg_id = kernel.execute('get_datasets()', callbacks);
}

function getRequestedDataset(dataset_name){
    var kernel = IPython.notebook.kernel;
    var callbacks = {
        iopub : {
             output : handle_dataset_retrieval,
        }
    }
    var msg_id = kernel.execute('get_dataset("'+dataset_name+'")', callbacks); 
}

function readRequestedDataset(dataset_uri){
    var kernel = IPython.notebook.kernel;
    var callbacks = {
        iopub : {
             output : handle_dataset_loading,
        }
    }
    var msg_id = kernel.execute('read_dataset_fromMetadata("'+dataset_uri+'")', callbacks);
    divVisibility("null-filterType");
    document.getElementById("filter-application-buttons").style.display = "block"; //because now we have a tempDF
    visibleDivId = null;
    applied_filters = {}
    filter_count = 0
    $("#show-filters").empty()
    successfulFilters = [];
    successfulFiltersPermanent = [];
}

</script>

#### Temp Dataset Preview (first 40 rows)

In [ ]:
#P4

if tempDF is not None:
    tempDF.show(40)
else:
    print("------------- empty ------------------")

#### Master Dataset Preview (first  20 rows)

In [ ]:
#P6

if masterDF is not None:
    masterDF.show(20)
else:
    print("------------- empty ------------------")